<h1><center>Laboratorio 6: La solicitud de Sergio 🤗</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Ignacio Reyes Oyarzún


### **Link de repositorio de GitHub:** [Repositorio - TR](https://github.com/TomiReyes/MDS7202-TR)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers.
- Utilizar diferentes algoritmos de cluster y ver el desempeño.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar clusters.
- Familiarizarse con plotly.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio

<center>
<img src="https://i.pinimg.com/originals/5a/a6/af/5aa6afde8490da403a21601adf7a7240.gif" width=400 />

En el corazón de las operaciones de Aerolínea Lucero, Sergio, el gerente de análisis de datos, reunió a un talentoso equipo de jóvenes científicos de datos para un desafío crucial: segmentar la base de datos de los clientes. “Nuestro objetivo es descubrir patrones en el comportamiento de los pasajeros que nos permitan personalizar servicios y optimizar nuestras campañas de marketing,” explicó Sergio, mientras desplegaba un amplio rango de datos que incluían desde hábitos de compra hasta opiniones sobre los vuelos.

Sergio encargó a los científicos de datos la tarea de aplicar técnicas avanzadas de clustering para identificar distintos segmentos de clientes, como los viajeros frecuentes y aquellos que eligen la aerolínea para celebrar ocasiones especiales. La meta principal era entender profundamente cómo estos grupos perciben la calidad y satisfacción de los servicios ofrecidos por la aerolínea.

A través de un enfoque meticuloso y colaborativo, los científicos de datos se abocaron a la tarea, buscando transformar los datos brutos en valiosos insights que permitirían a Aerolínea Lucero no solo mejorar su servicio, sino también fortalecer las relaciones con sus clientes mediante una oferta más personalizada y efectiva.

## Importamos librerias utiles 😸

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.subplots as sp
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import time


from sklearn.preprocessing import MinMaxScaler
import plotly.figure_factory as ff
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

## 1. Estudio de Performance 📈 [10 Puntos]



<center>
<img src="https://user-images.githubusercontent.com/57133330/188281408-c67df9ee-fd1f-4b37-833b-f02848f1ce02.gif" width=300>

Don Sergio les ha encomendado su primera tarea: analizar diversas técnicas de clustering. Su objetivo es entender detalladamente cómo funcionan estos métodos en términos de segmentación y eficiencia en tiempo de ejecución.

Analice y compare el desempeño, tiempo de ejecución y visualizaciones de cuatro algoritmos de clustering (k-means, DBSCAN, Ward y GMM) aplicados a tres conjuntos de datos, incrementando progresivamente su tamaño. Utilice Plotly para las gráficas y discuta los resultados tanto cualitativa como cuantitativamente.

Uno de los requisitos establecidos por Sergio es que el análisis se lleve a cabo utilizando Plotly; de no ser así, se considerará incorrecto. Para facilitar este proceso, se ha proporcionado un código de Plotly que puede servir como base para realizar las gráficas. Apóyese en el código entregado para efectuar el análisis y tome como referencia la siguiente imagen para realizar los gráficos:

<img src='https://gitlab.com/imezadelajara/datos_clase_7_mds7202/-/raw/main/misc_images/Screenshot_2024-04-26_at_9.10.44_AM.png' width=800 />

En el gráfico se visualizan en dos dimensiones los diferentes tipos de datos proporcionados en `datasets`. Cada columna corresponde a un modelo de clustering diferente, mientras que cada fila representa un conjunto de datos distinto. Cada uno de los gráficos incluye el tiempo en segundos que tarda el análisis y la métrica Silhouette obtenida.

Para ser más específicos, usted debe cumplir los siguientes objetivos:
1. Generar una función que permita replicar el gráfico expuesto en la imagen (no importa que los colores calcen). [4 puntos]
2. Ejecuta la función para un `n_samples` igual a 1000, 5000, 10000. [2 puntos]
3. Analice y compare el desempeño, tiempo de ejecución y visualizaciones de cuatro algoritmos de clustering utilizando las 3 configuraciones dadas en `n_samples`. [4 puntos]


> ❗ Tiene libertad absoluta de escoger los hiper parámetros de los cluster, sin embargo, se recomienda verificar el dominio de las variables para realizar la segmentación.

> ❗ Recuerde que es obligatorio el uso de plotly.


In [3]:
"""
En la siguiente celda se crean los datos ficticios a usar en la sección 1 del lab.
❗No realice cambios a esta celda a excepción de n_samples❗
"""

# Datos a utilizar


def create_data(n_samples):
    # Lunas
    moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=30)
    # Blobs
    blobs = datasets.make_blobs(n_samples=n_samples, random_state=172)
    # Datos desiguales
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    mutated = (np.dot(blobs[0], transformation), blobs[1])

    # Generamos Dataset
    dataset = {
        "moons": {"x": moons[0], "classes": moons[1], "n_cluster": 2},
        "blobs": {"x": blobs[0], "classes": blobs[1], "n_cluster": 3},
        "mutated": {"x": mutated[0], "classes": mutated[1], "n_cluster": 3},
    }
    return dataset

**Respuestas:**

In [4]:
def plot_scatter(x, y, color, row, col, fig):
    scatter = go.Scatter(
        x=x,
        y=y,
        mode="markers",
        marker=dict(color=color, colorscale="Viridis", line=dict(width=0.5)),
        showlegend=False,
    )
    fig.add_trace(scatter, row=row, col=col)


In [5]:
def plot_all_clustering_results(datasets):
    # Moons
    x_moons = datasets["moons"]["x"]
    n_clusters_moons = datasets["moons"]["n_cluster"]

    # K-means
    start_time_moons_kmeans = time.time()
    kmeans_moons = KMeans(n_clusters=n_clusters_moons, random_state=10)
    labels_moons_kmeans = kmeans_moons.fit_predict(x_moons)
    exec_time_moons_kmeans = time.time() - start_time_moons_kmeans
    silhouette_moons_kmeans = silhouette_score(x_moons, labels_moons_kmeans)

    # GMM
    start_time_moons_gmm = time.time()
    gmm_moons = GaussianMixture(n_components=n_clusters_moons, random_state=10)
    labels_moons_gmm = gmm_moons.fit(x_moons).predict(x_moons)
    exec_time_moons_gmm = time.time() - start_time_moons_gmm
    silhouette_moons_gmm = silhouette_score(x_moons, labels_moons_gmm)

    # Ward
    start_time_moons_ward = time.time()
    ward_moons = AgglomerativeClustering(n_clusters=n_clusters_moons, linkage="ward")
    labels_moons_ward = ward_moons.fit_predict(x_moons)
    exec_time_moons_ward = time.time() - start_time_moons_ward
    silhouette_moons_ward = silhouette_score(x_moons, labels_moons_ward)

    # DBSCAN
    start_time_moons_dbscan = time.time()
    dbscan_moons = DBSCAN(eps=0.5, min_samples=5)
    labels_moons_dbscan = dbscan_moons.fit_predict(x_moons)
    exec_time_moons_dbscan = time.time() - start_time_moons_dbscan
    silhouette_moons_dbscan = (
        silhouette_score(x_moons, labels_moons_dbscan)
        if len(set(labels_moons_dbscan)) > 1
        else -1
    )

    # Blobs
    x_blobs = datasets["blobs"]["x"]
    n_clusters_blobs = datasets["blobs"]["n_cluster"]

    # K-means
    start_time_blobs_kmeans = time.time()
    kmeans_blobs = KMeans(n_clusters=n_clusters_blobs, random_state=10)
    labels_blobs_kmeans = kmeans_blobs.fit_predict(x_blobs)
    exec_time_blobs_kmeans = time.time() - start_time_blobs_kmeans
    silhouette_blobs_kmeans = silhouette_score(x_blobs, labels_blobs_kmeans)

    # GMM
    start_time_blobs_gmm = time.time()
    gmm_blobs = GaussianMixture(n_components=n_clusters_blobs, random_state=10)
    labels_blobs_gmm = gmm_blobs.fit(x_blobs).predict(x_blobs)
    exec_time_blobs_gmm = time.time() - start_time_blobs_gmm
    silhouette_blobs_gmm = silhouette_score(x_blobs, labels_blobs_gmm)

    # Ward
    start_time_blobs_ward = time.time()
    ward_blobs = AgglomerativeClustering(n_clusters=n_clusters_blobs, linkage="ward")
    labels_blobs_ward = ward_blobs.fit_predict(x_blobs)
    exec_time_blobs_ward = time.time() - start_time_blobs_ward
    silhouette_blobs_ward = silhouette_score(x_blobs, labels_blobs_ward)

    # DBSCAN
    start_time_blobs_dbscan = time.time()
    dbscan_blobs = DBSCAN(eps=0.5, min_samples=5)
    labels_blobs_dbscan = dbscan_blobs.fit_predict(x_blobs)
    exec_time_blobs_dbscan = time.time() - start_time_blobs_dbscan
    silhouette_blobs_dbscan = (
        silhouette_score(x_blobs, labels_blobs_dbscan)
        if len(set(labels_blobs_dbscan)) > 1
        else -1
    )

    # Mutated
    x_mutated = datasets["mutated"]["x"]
    n_clusters_mutated = datasets["mutated"]["n_cluster"]

    # K-means
    start_time_mutated_kmeans = time.time()
    kmeans_mutated = KMeans(n_clusters=n_clusters_mutated, random_state=10)
    labels_mutated_kmeans = kmeans_mutated.fit_predict(x_mutated)
    exec_time_mutated_kmeans = time.time() - start_time_mutated_kmeans
    silhouette_mutated_kmeans = silhouette_score(x_mutated, labels_mutated_kmeans)

    # GMM
    start_time_mutated_gmm = time.time()
    gmm_mutated = GaussianMixture(n_components=n_clusters_mutated, random_state=10)
    labels_mutated_gmm = gmm_mutated.fit(x_mutated).predict(x_mutated)
    exec_time_mutated_gmm = time.time() - start_time_mutated_gmm
    silhouette_mutated_gmm = silhouette_score(x_mutated, labels_mutated_gmm)

    # Ward
    start_time_mutated_ward = time.time()
    ward_mutated = AgglomerativeClustering(
        n_clusters=n_clusters_mutated, linkage="ward"
    )
    labels_mutated_ward = ward_mutated.fit_predict(x_mutated)
    exec_time_mutated_ward = time.time() - start_time_mutated_ward
    silhouette_mutated_ward = silhouette_score(x_mutated, labels_mutated_ward)

    # DBSCAN
    start_time_mutated_dbscan = time.time()
    dbscan_mutated = DBSCAN(eps=0.5, min_samples=5)
    labels_mutated_dbscan = dbscan_mutated.fit_predict(x_mutated)
    exec_time_mutated_dbscan = time.time() - start_time_mutated_dbscan
    silhouette_mutated_dbscan = (
        silhouette_score(x_mutated, labels_mutated_dbscan)
        if len(set(labels_mutated_dbscan)) > 1
        else -1
    )

    fig = sp.make_subplots(
        rows=3,
        cols=4,
        subplot_titles=[
            f"{exec_time_moons_kmeans:.2f}s, {silhouette_moons_kmeans:.2f}",
            f"{exec_time_moons_gmm:.2f}s, {silhouette_moons_gmm:.2f}",
            f"{exec_time_moons_ward:.2f}s,{silhouette_moons_ward:.2f}",
            f"{exec_time_moons_dbscan:.2f}s, {silhouette_moons_dbscan:.2f}",
            f"{exec_time_blobs_kmeans:.2f}s, {silhouette_blobs_kmeans:.2f}",
            f"{exec_time_blobs_gmm:.2f}s, {silhouette_blobs_gmm:.2f}",
            f"{exec_time_blobs_ward:.2f}s, {silhouette_blobs_ward:.2f}",
            f"{exec_time_blobs_dbscan:.2f}s, {silhouette_blobs_dbscan:.2f}",
            f"{exec_time_mutated_kmeans:.2f}s, {silhouette_mutated_kmeans:.2f}",
            f"{exec_time_mutated_gmm:.2f}s, {silhouette_mutated_gmm:.2f}",
            f"{exec_time_mutated_ward:.2f}s, {silhouette_mutated_ward:.2f}",
            f"{exec_time_mutated_dbscan:.2f}s, {silhouette_mutated_dbscan:.2f}",
        ],
        horizontal_spacing=0.05,
        vertical_spacing=0.1,
    )

    plot_scatter(
        x_moons[:, 0],
        x_moons[:, 1],
        labels_moons_kmeans,
        1,
        1,
        fig,
    )
    plot_scatter(
        x_moons[:, 0],
        x_moons[:, 1],
        labels_moons_gmm,
        1,
        2,
        fig,
    )
    plot_scatter(
        x_moons[:, 0],
        x_moons[:, 1],
        labels_moons_ward,
        1,
        3,
        fig,
    )
    plot_scatter(
        x_moons[:, 0],
        x_moons[:, 1],
        labels_moons_dbscan,
        1,
        4,
        fig,
    )

    # Blobs
    plot_scatter(
        x_blobs[:, 0],
        x_blobs[:, 1],
        labels_blobs_kmeans,
        2,
        1,
        fig,
    )
    plot_scatter(
        x_blobs[:, 0],
        x_blobs[:, 1],
        labels_blobs_gmm,
        2,
        2,
        fig,
    )
    plot_scatter(
        x_blobs[:, 0],
        x_blobs[:, 1],
        labels_blobs_ward,
        2,
        3,
        fig,
    )
    plot_scatter(
        x_blobs[:, 0],
        x_blobs[:, 1],
        labels_blobs_dbscan,
        2,
        4,
        fig,
    )

    # Mutated
    plot_scatter(
        x_mutated[:, 0],
        x_mutated[:, 1],
        labels_mutated_kmeans,
        3,
        1,
        fig,
    )
    plot_scatter(
        x_mutated[:, 0],
        x_mutated[:, 1],
        labels_mutated_gmm,
        3,
        2,
        fig,
    )
    plot_scatter(
        x_mutated[:, 0],
        x_mutated[:, 1],
        labels_mutated_ward,
        3,
        3,
        fig,
    )
    plot_scatter(
        x_mutated[:, 0],
        x_mutated[:, 1],
        labels_mutated_dbscan,
        3,
        4,
        fig,
    )

    fig.update_layout(
        height=600,
        width=800,
        title_x=0.5,
    )
    fig.show()


In [ ]:
n_samples = 1000
data = create_data(n_samples)
plot_all_clustering_results(data)

In [ ]:
# Crear los datos y graficar los resultados
n_samples = 5000
data = create_data(n_samples)
plot_all_clustering_results(data)

In [ ]:
# Crear los datos y graficar los resultados
n_samples = 10000
data = create_data(n_samples)
plot_all_clustering_results(data)

3. 

Para todos los casos el primer gráfico de DBScan tiene malos resultados, dado que no distingue. 
En cuanto a resultados según los gráficos y la métrica de Silhoutte, DBSCAN tiene los peores resultados. 
Por lo general, WARD es el que toma más tiempo de ejecución respecto a los otros algoritmos. 
En cuanto a la valoración de resultados en función de los tiempos de ejecución, GMM es el mejor algoritmo, para estos datos y esta configuración de parámetros.



## 2. Análisis de Satisfacción de Vuelos. [10 puntos]

<center>
<img src="https://i.gifer.com/2Hci.gif" width=400 />

Habiendo entendido cómo funcionan los modelos de aprendizaje no supervisado, *Don Sergio* le encomienda estudiar la satisfacción de pasajeros al haber tomado un vuelo en alguna de sus aerolineas. Para esto, el magnate le dispone del dataset `aerolineas_licer.parquet`, el cual contiene el grado de satisfacción de los clientes frente a diferentes aspectos del vuelo. Las características del vuelo se definen a continuación:

- *Gender*: Género de los pasajeros (Femenino, Masculino)
- *Customer Type*: Tipo de cliente (Cliente habitual, cliente no habitual)
- *Age*: Edad actual de los pasajeros
- *Type of Travel*: Propósito del vuelo de los pasajeros (Viaje personal, Viaje de negocios)
- *Class*: Clase de viaje en el avión de los pasajeros (Business, Eco, Eco Plus)
- *Flight distance*: Distancia del vuelo de este viaje
- *Inflight wifi service*: Nivel de satisfacción del servicio de wifi durante el vuelo (0:No Aplicable; 1-5)
- *Departure/Arrival time convenient*: Nivel de satisfacción con la conveniencia del horario de salida/llegada
- *Ease of Online booking*: Nivel de satisfacción con la facilidad de reserva en línea
- *Gate location*: Nivel de satisfacción con la ubicación de la puerta
- *Food and drink*: Nivel de satisfacción con la comida y la bebida
- *Online boarding*: Nivel de satisfacción con el embarque en línea
- *Seat comfort*: Nivel de satisfacción con la comodidad del asiento
- *Inflight entertainment*: Nivel de satisfacción con el entretenimiento durante el vuelo
- *On-board service*: Nivel de satisfacción con el servicio a bordo
- *Leg room service*: Nivel de satisfacción con el espacio para las piernas
- *Baggage handling*: Nivel de satisfacción con el manejo del equipaje
- *Check-in service*: Nivel de satisfacción con el servicio de check-in
- *Inflight service*: Nivel de satisfacción con el servicio durante el vuelo
- *Cleanliness*: Nivel de satisfacción con la limpieza
- *Departure Delay in Minutes*: Minutos de retraso en la salida
- *Arrival Delay in Minutes*: Minutos de retraso en la llegada

En consideración de lo anterior, realice las siguientes tareas:

0. Ingeste el dataset a su ambiente de trabajo.

1. Seleccione **sólo las variables numéricas del dataset**.  Explique qué éfectos podría causar el uso de variables categóricas en un algoritmo no supervisado. [2 punto]

2. Realice una visualización de la distribución de cada variable y analice cada una de estas distribuciones. [2 punto]

3. Basándose en los gráficos, evalúe la necesidad de escalar los datos y explique el motivo de su decisión. [2 puntos]

4. Examine la correlación entre las variables mediante un correlograma. [2 puntos]

5. De acuerdo con los resultados obtenidos en 5, reduzca la dimensionalidad del conjunto de datos a cuatro variables, justificando su elección respecto a las variables que decide eliminar. [2 puntos]

**Respuesta:**

In [ ]:
df = pd.read_parquet("aerolineas_lucer.parquet")
df.head(2)

In [ ]:
var_num = df.select_dtypes(include=["number"])
var_num = var_num.drop(columns=["id"])
var_num.dtypes

En un algoritmo no supervisado las variables categóricas pueden tener efectos negativos porque, generalmente, los algoritmos usan medidas de distancia para establecer a que cluster pertenece cada dato. En ese caso, con variables que no pueden cumplir esta condición no se pueden calcular de buena manera.

In [ ]:
fig1 = px.histogram(
    var_num,
    x=var_num.columns[0],
    nbins=30,
    title=f"Distribución de {var_num.columns[0]}",
)
fig1.show()

fig2 = px.histogram(
    var_num,
    x=var_num.columns[1],
    nbins=30,
    title=f"Distribución de {var_num.columns[1]}",
)
fig2.show()

fig3 = px.histogram(
    var_num,
    x=var_num.columns[2],
    nbins=30,
    title=f"Distribución de {var_num.columns[2]}",
)
fig3.show()

fig4 = px.histogram(
    var_num,
    x=var_num.columns[3],
    nbins=30,
    title=f"Distribución de {var_num.columns[3]}",
)
fig4.show()

fig5 = px.histogram(
    var_num,
    x=var_num.columns[4],
    nbins=30,
    title=f"Distribución de {var_num.columns[4]}",
)
fig5.show()

fig6 = px.histogram(
    var_num,
    x=var_num.columns[5],
    nbins=30,
    title=f"Distribución de {var_num.columns[5]}",
)
fig6.show()

fig7 = px.histogram(
    var_num,
    x=var_num.columns[6],
    nbins=30,
    title=f"Distribución de {var_num.columns[6]}",
)
fig7.show()

fig8 = px.histogram(
    var_num,
    x=var_num.columns[7],
    nbins=30,
    title=f"Distribución de {var_num.columns[7]}",
)
fig8.show()

fig9 = px.histogram(
    var_num,
    x=var_num.columns[8],
    nbins=30,
    title=f"Distribución de {var_num.columns[8]}",
)
fig9.show()

fig10 = px.histogram(
    var_num,
    x=var_num.columns[9],
    nbins=30,
    title=f"Distribución de {var_num.columns[9]}",
)
fig10.show()

fig11 = px.histogram(
    var_num,
    x=var_num.columns[10],
    nbins=30,
    title=f"Distribución de {var_num.columns[10]}",
)
fig11.show()

fig12 = px.histogram(
    var_num,
    x=var_num.columns[11],
    nbins=30,
    title=f"Distribución de {var_num.columns[11]}",
)
fig12.show()

fig13 = px.histogram(
    var_num,
    x=var_num.columns[12],
    nbins=30,
    title=f"Distribución de {var_num.columns[12]}",
)
fig13.show()

fig14 = px.histogram(
    var_num,
    x=var_num.columns[13],
    nbins=30,
    title=f"Distribución de {var_num.columns[13]}",
)
fig14.show()

fig15 = px.histogram(
    var_num,
    x=var_num.columns[14],
    nbins=30,
    title=f"Distribución de {var_num.columns[14]}",
)
fig15.show()

fig16 = px.histogram(
    var_num,
    x=var_num.columns[15],
    nbins=30,
    title=f"Distribución de {var_num.columns[15]}",
)
fig16.show()

fig17 = px.histogram(
    var_num,
    x=var_num.columns[16],
    nbins=30,
    title=f"Distribución de {var_num.columns[16]}",
)
fig17.show()

fig18 = px.histogram(
    var_num,
    x=var_num.columns[17],
    nbins=30,
    title=f"Distribución de {var_num.columns[17]}",
)
fig18.show()


En particular hay 2 variables que tienen valores distintos al resto, las variables de delay in minutes no tienen valores categoricos de a 1 a 5 por lo que su representación y valoración en un modelo es distinto. Visto de esta forma, es necesario escalar los datos. Viendo las variables: La edad cumple casi con una distribución normal alrededor de los 40 años, el flight distance se concentra en menos de 10.000, casi todas las variables categóricas se concentran entre 3, 4 o 5, y los delay son menores de 200 casi todos. 

In [12]:
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(var_num)

df_scaled = pd.DataFrame(scaled_array, columns=var_num.columns, index=var_num.index)

In [ ]:
matriz_correlacion = df_scaled.corr()

fig = ff.create_annotated_heatmap(
    z=matriz_correlacion.values,
    x=list(matriz_correlacion.columns),
    y=list(matriz_correlacion.columns),
    annotation_text=np.round(matriz_correlacion.values, 2),
    colorscale="Viridis",
)
fig.update_layout(title="Correlograma")
fig.show()

5. De acuerdo con los resultados obtenidos en 4), reduzca la dimensionalidad del conjunto de datos a cuatro variables, justificando su elección respecto a las variables que decide eliminar. [2 puntos]

En vista de que hay variables que están muy correlacionadas, se seleccionan las siguientes:
- Age, porque segmenta bien a los clientes y tiene poca correlación con el resto de variables 
- Departure Delay in Minutes, porque con la otra variable de delay son identicas y son muy distintas al resto 
- Checkin service, porque tiene baja correlación con las demás variables y puede tener más impacto que otras variables poca correlacionadas (era esta o Flight Distance)
- Inflight entertainment, porque está muy correlacionada con hartas variables, pero poco correlacionada con las otras 3 variables elegidas, así que representa bien a las no elegidas

In [ ]:
df_importante = df_scaled[
    ["Age", "Departure Delay in Minutes", "Checkin service", "Inflight entertainment"]
]
df_importante.head()

## 3. Preprocesamiento 🎭. [10 puntos]

<center>
<img src="https://i.pinimg.com/originals/1e/a8/0e/1ea80e7cea0d429146580c7e91c5b944.gif" width=400>

Tras quedar satisfecho con los resultados presentados en el punto 2, el dueño de la empresa ha solicitado que se preprocesen los datos mediante un `pipeline`. Es crucial que este proceso tenga en cuenta las observaciones derivadas de los análisis anteriores. Adicionalmente, ha expresado su interés en visualizar el conjunto de datos en un gráfico de dos o tres dimensiones.

Basándose en los análisis realizados anteriormente:
1. Cree un `pipeline` que incluya PCA, utilizando las consideraciones mencionadas previamente para proyectar los datos a dos dimensiones. [4 puntos]
2. Grafique los resultados obtenidos y comente lo visualizado. [6 puntos]

**Respuestas:**

In [15]:
pipeline = Pipeline([("scaler", StandardScaler()), ("pca", PCA(n_components=2))])

pca_result = pipeline.fit_transform(df_importante)


In [ ]:
pca_df = pd.DataFrame(pca_result, columns=["PCA1", "PCA2"])

fig = px.scatter(
    pca_df,
    x="PCA1",
    y="PCA2",
    title="Proyección PCA a 2 Dimensiones",
    labels={"PCA1": "Componente Principal 1", "PCA2": "Componente Principal 2"},
)
fig.show()

## 4. Outliers 🚫🙅‍♀️❌🙅‍♂️ [10 puntos]

<center>
<img src="https://joachim-gassen.github.io/images/ani_sim_bad_leverage.gif" width=250>

Con el objetivo de mantener la claridad en su análisis, Don Sergio le ha solicitado entrenar un modelo que identifique pasajeros con comportamientos altamente atípicos.

1. Utilice `IsolationForest` para clasificar las anomalías del dataset (sin aplicar PCA), configurando el modelo para que sólo el 1% de los datos sean considerados anómalos. Asegúrese de integrar esta tarea dentro de un `pipeline`. [3 puntos]

2. Visualice los resultados en el gráfico de dos dimensiones previamente creado. [3 puntos]

3. ¿Cómo evaluaría el rendimiento de su modelo en la detección de anomalías? [4 puntos]

**Respuestas:**

In [17]:
important_data_array = df_importante.to_numpy()

pipeline_anomaly = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("isolation_forest", IsolationForest(contamination=0.01, random_state=10)),
    ]
)

pipeline_anomaly.fit(important_data_array)

anomalia_casos = pipeline_anomaly.named_steps["isolation_forest"].predict(
    important_data_array
)

In [ ]:
pca_df["anomaly"] = anomalia_casos

fig = px.scatter(
    pca_df,
    x="PCA1",
    y="PCA2",
    color="anomaly",
    color_discrete_map={1: "blue", -1: "red"},
    title="Proyección PCA con Detección de Anomalías (IsolationForest)",
    labels={
        "PCA1": "Componente Principal 1",
        "PCA2": "Componente Principal 2",
        "anomaly": "Anomalía",
    },
)
fig.show()


Sin saber si apliqué todo bien, lo calificaría mal, ya que gráficamente no está mostrando como anomalías a los datos más distantes, cuando si debería ser el caso.

## 5. Métricas de Desempeño 🚀 [10 puntos]

<center>
<img src="https://giffiles.alphacoders.com/219/219081.gif" width=300>

Motivado por incrementar su fortuna, Don Sergio le solicita entrenar un modelo que le permita segmentar a los pasajeros en grupos distintos, con el objetivo de optimizar las diversas campañas de marketing diseñadas por su equipo. Para ello, le se pide realizar las siguientes tareas:

1. Utilizar el modelo **Gaussian Mixture** y explore diferentes configuraciones de número de clústers, específicamente entre 3 y 8. Asegúrese de integrar esta operación dentro de un `pipeline`. [4 puntos]
2. Explique cuál sería el criterio adecuado para seleccionar el número óptimo de clústers. **Justifique de forma estadistica y a traves de gráficos.** [6 puntos]

> **HINT:** Se recomienda investigar sobre los criterios AIC y BIC para esta tarea.

**Respuestas:**

In [19]:
pipeline_gmm = Pipeline(
    [("scaler", StandardScaler()), ("gmm", GaussianMixture(random_state=10))]
)

In [20]:
important_data_array = df_importante.to_numpy()

# n_clusters=3
pipeline_gmm_3 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=3, random_state=10)),
    ]
)
pipeline_gmm_3.fit(important_data_array)
gmm_3 = pipeline_gmm_3.named_steps["gmm"]
aic_3 = gmm_3.aic(important_data_array)
bic_3 = gmm_3.bic(important_data_array)

# n_clusters=4
pipeline_gmm_4 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=4, random_state=10)),
    ]
)
pipeline_gmm_4.fit(important_data_array)
gmm_4 = pipeline_gmm_4.named_steps["gmm"]
aic_4 = gmm_4.aic(important_data_array)
bic_4 = gmm_4.bic(important_data_array)

# n_clusters=5
pipeline_gmm_5 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=5, random_state=10)),
    ]
)
pipeline_gmm_5.fit(important_data_array)
gmm_5 = pipeline_gmm_5.named_steps["gmm"]
aic_5 = gmm_5.aic(important_data_array)
bic_5 = gmm_5.bic(important_data_array)

# n_clusters=6
pipeline_gmm_6 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=6, random_state=10)),
    ]
)
pipeline_gmm_6.fit(important_data_array)
gmm_6 = pipeline_gmm_6.named_steps["gmm"]
aic_6 = gmm_6.aic(important_data_array)
bic_6 = gmm_6.bic(important_data_array)

# n_clusters=7
pipeline_gmm_7 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=7, random_state=10)),
    ]
)
pipeline_gmm_7.fit(important_data_array)
gmm_7 = pipeline_gmm_7.named_steps["gmm"]
aic_7 = gmm_7.aic(important_data_array)
bic_7 = gmm_7.bic(important_data_array)

# n_clusters=8
pipeline_gmm_8 = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("gmm", GaussianMixture(n_components=8, random_state=10)),
    ]
)
pipeline_gmm_8.fit(important_data_array)
gmm_8 = pipeline_gmm_8.named_steps["gmm"]
aic_8 = gmm_8.aic(important_data_array)
bic_8 = gmm_8.bic(important_data_array)


In [ ]:
n_clusters = [3, 4, 5, 6, 7, 8]
aic_scores = [aic_3, aic_4, aic_5, aic_6, aic_7, aic_8]
bic_scores = [bic_3, bic_4, bic_5, bic_6, bic_7, bic_8]

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=n_clusters,
        y=aic_scores,
        mode="lines+markers",
        name="AIC",
        line=dict(color="blue"),
        marker=dict(size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=n_clusters,
        y=bic_scores,
        mode="lines+markers",
        name="BIC",
        line=dict(color="red"),
        marker=dict(size=10),
    )
)

fig.update_layout(
    title="AIC y BIC ",
    xaxis_title="Número de Clusters",
    yaxis_title="AIC / BIC",
    legend_title="Criterios",
    height=500,
    width=800,
)

fig.show()


## 6. Análisis de resultados 📊 [10 puntos]

<center>
<img src="https://i.gifer.com/7wTk.gif" width=300>

Una vez identificado el número óptimo de clústers, se le pide realizar lo siguiente:

1. Utilizar la proyección en dos dimensiones para visualizar cada clúster claramente. [2 puntos]

2. ¿Es posible distinguir claramente entre los clústers generados? [2 puntos]

3. Proporcionar una descripción breve de cada clúster utilizando estadísticas descriptivas básicas, como la media y la desviación estándar, para resumir las características de las variables utilizadas en estos algoritmos. [2 puntos]

4. Proceda a visualizar los clústers en tres dimensiones para una perspectiva más detallada. [2 puntos]

5. ¿Cómo afecta esto a sus conclusiones anteriores? [2 puntos]

**Respuestas:**

In [ ]:
pipeline_gmm_7.fit(df_importante)
gmm_7 = pipeline_gmm_7.named_steps["gmm"]
cluster_labels = gmm_7.predict(df_importante)

pca_result = pipeline.named_steps["pca"].transform(df_importante)

pca_df = pd.DataFrame(pca_result, columns=["PCA1", "PCA2"])
pca_df["cluster"] = cluster_labels

fig = px.scatter(
    pca_df,
    x="PCA1",
    y="PCA2",
    color="cluster",
    title="Visualización de Clústers en 2D",
    labels={
        "PCA1": "Componente Principal 1",
        "PCA2": "Componente Principal 2",
        "cluster": "Cluster",
    },
)
fig.show()

2. 
Es posible distinguir clusters, pero no de forma clara, ya que hay muchos datos concentrados en un cluster y este se podría segmentar un más. De todos modos, no hay datos mezclados dentro de los clusters y eso es bueno.

In [ ]:
df_importante["cluster"] = cluster_labels

cluster_stats = df_importante.groupby("cluster").agg(["mean", "std"])
cluster_stats

3. 

Viendo las medias y desviaciones estandar, los clusters se pueden diferenciar, particularmente viendo las medias. Ya las desviación estandar indican que hay variables que no están tan concentradas en la media, como el caso de Departure Delay in Minutes. Por lo demas, en las otras variables si se puede considerar que hay diferencias entre ambos casos. 

In [ ]:
pca_3d = PCA(n_components=3)
pca_result_3d = pca_3d.fit_transform(
    StandardScaler().fit_transform(df_importante.drop(columns="cluster"))
)

pca_3d_df = pd.DataFrame(pca_result_3d, columns=["PCA1", "PCA2", "PCA3"])
pca_3d_df["cluster"] = cluster_labels

fig = px.scatter_3d(
    pca_3d_df,
    x="PCA1",
    y="PCA2",
    z="PCA3",
    color="cluster",
    title="Visualización de Clústers en 3D",
    labels={
        "PCA1": "Componente Principal 1",
        "PCA2": "Componente Principal 2",
        "PCA3": "Componente Principal 3",
        "cluster": "Cluster",
    },
)
fig.show()

5. 
En una visualización de 3 dimensiones de los clústers, se puede observar que los clusters están mejor separados, al menos es más clara la diferencia. 
Esto da a entender que el modelo de Gaussian Mixture ha sido capaz de identificar patrones en los datos que permiten agruparlos de manera efectiva.